# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

Setup the spark environ

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession


pull in needed packages

In [2]:
from pyspark.sql.functions import when, col

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import when, col
from pyspark.ml.regression import LinearRegression



create the spark session

In [3]:
spark = SparkSession.builder.appName("LRCruise").getOrCreate()

Two local functions for hot encoding and automated training/test splits

In [24]:
def encode_cruiseline(df,cruiseline):
    # withColumn('doubleage',df['age']*2).show()
    df = df.withColumn(cruiseline, when(col("Cruise_line") == cruiseline,1).otherwise(0) )
    return df


def progressive_challenges(rshp_df,train_sizes):
    lr = LinearRegression(labelCol='crew')
    rmse = []
    rms = []
    r2 = []
    for train_size in train_sizes:
        test_size = 1.00 - train_size
        train_df,test_df = rshp_df.randomSplit([train_size,test_size])
        lrModel = lr.fit(train_df,)
        results = lrModel.evaluate(test_df)
        rmse.append(results.rootMeanSquaredError)
        rms.append(results.meanSquaredError)
        r2.append(results.r2)
    return rmse,rms, r2

In [5]:
df = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [6]:
df.select("Cruise_line").distinct().show()

+-----------------+
|      Cruise_line|
+-----------------+
|            Costa|
|              P&O|
|           Cunard|
|Regent_Seven_Seas|
|              MSC|
|         Carnival|
|          Crystal|
|           Orient|
|         Princess|
|        Silversea|
|         Seabourn|
| Holland_American|
|         Windstar|
|           Disney|
|        Norwegian|
|          Oceania|
|          Azamara|
|        Celebrity|
|             Star|
|  Royal_Caribbean|
+-----------------+



In [7]:
# Hot encode the cruise line
for crln in ["Costa","P&O","Cunard","Regent_Seven_Seas","MSC","Carnival","Crystal",
             "Orient","Princess","Silversea","Seabourn","Holland_American","Windstar",
             "Disney","Norwegian","Oceania","Azamara","Celebrity","Star","Royal_Caribbean"]:
    df = encode_cruiseline(df,crln)

In [8]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Costa: integer (nullable = false)
 |-- P&O: integer (nullable = false)
 |-- Cunard: integer (nullable = false)
 |-- Regent_Seven_Seas: integer (nullable = false)
 |-- MSC: integer (nullable = false)
 |-- Carnival: integer (nullable = false)
 |-- Crystal: integer (nullable = false)
 |-- Orient: integer (nullable = false)
 |-- Princess: integer (nullable = false)
 |-- Silversea: integer (nullable = false)
 |-- Seabourn: integer (nullable = false)
 |-- Holland_American: integer (nullable = false)
 |-- Windstar: integer (nullable = false)
 |-- Disney: integer (nullable = false)
 |-- Norwegian: integer (nullable = false)
 |--

In [9]:
# Create the feature vector

fields = ["Age","Tonnage","passengers","length","cabins","passenger_density","Costa","P&O","Cunard","Regent_Seven_Seas","MSC","Carnival","Crystal",
             "Orient","Princess","Silversea","Seabourn","Holland_American","Windstar",
             "Disney","Norwegian","Oceania","Azamara","Celebrity","Star","Royal_Caribbean"]

assembler = VectorAssembler(
    inputCols=fields,
    outputCol="features")

rshp_df = assembler.transform(df)

In [10]:
rshp_df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Costa: integer (nullable = false)
 |-- P&O: integer (nullable = false)
 |-- Cunard: integer (nullable = false)
 |-- Regent_Seven_Seas: integer (nullable = false)
 |-- MSC: integer (nullable = false)
 |-- Carnival: integer (nullable = false)
 |-- Crystal: integer (nullable = false)
 |-- Orient: integer (nullable = false)
 |-- Princess: integer (nullable = false)
 |-- Silversea: integer (nullable = false)
 |-- Seabourn: integer (nullable = false)
 |-- Holland_American: integer (nullable = false)
 |-- Windstar: integer (nullable = false)
 |-- Disney: integer (nullable = false)
 |-- Norwegian: integer (nullable = false)
 |--

In [11]:
lr = LinearRegression(labelCol='crew')
lrModel = lr.fit(rshp_df,)

In [12]:
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [0.00727114555098389,0.014577845490114599,-0.09791674271221261,0.48507139216942013,0.7248614404544313,0.0007399768126846898,-0.3496383874645716,-0.015606001134954655,0.08605663107524626,0.6450614886447045,-0.05459002766437478,0.2761014942290554,0.24801265864105193,-0.3563577293486538,0.01738310614488004,0.3175009892704016,0.4695359123716959,-0.4814531633982071,-0.185340944919023,0.7017943718835417,0.3743422682973946,0.6409611127666073,0.16831640353715727,0.5747347281459246,1.3773007777549091,-1.1325478935805386] Intercept: -1.909903171781209


In [13]:
results = lrModel.evaluate(rshp_df)

In [14]:
results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-6.13293575923989...|
|-6.13293575923989...|
|-6.50660306755312...|
|   6.526259657202212|
| -1.3109977854063501|
| 0.09096788169151893|
| 0.14186590054840487|
| 0.07316057564869283|
|  0.1127813183444708|
|-0.30771529272508147|
| -0.9495191549988728|
|-0.03181337747404...|
| 0.12005246389545476|
| 0.12732360944643695|
| -0.7193516858143063|
| -0.9349768638969049|
|  0.2578728189234081|
| 0.14186590054840487|
| -1.4888020581839747|
| 0.10551017279348684|
+--------------------+
only showing top 20 rows



In [15]:
print("RMSE: {}".format(results.rootMeanSquaredError))
print("MSE: {}".format(results.meanSquaredError))

RMSE: 0.8007646556078504
MSE: 0.6412240336707592


In [16]:
train_df,test_df = rshp_df.randomSplit([0.5,0.5])

lrModel = lr.fit(train_df,)
results = lrModel.evaluate(test_df)
print("RMSE: {}".format(results.rootMeanSquaredError))
print("MSE: {}".format(results.meanSquaredError))

RMSE: 0.6841987368755853
MSE: 0.46812791154214634


In [26]:

train_sizes = [0.1,0.1,0.1,0.1,0.1,
               0.3,0.3,0.3,0.3,0.3,
               0.5,0.5,0.5,0.5,0.5,
               0.7,0.7,0.7,0.7,0.7,
               0.9,0.9,0.9,0.9,0.9]
rmse, rms, r2 = progressive_challenges(rshp_df,train_sizes)

print(rmse)
print(rms)

[2.4770411063556703, 2.810334128560862, 2.6600835547567843, 1.1768935592143286, 1.5985963811009287, 1.0647792507208917, 1.5227618602873052, 1.0239021948016094, 1.641040856123727, 0.9452564775182413, 1.1225940300589132, 1.2758153210334473, 1.133726768796237, 1.0196293315942608, 0.8622852673188082, 1.2830930308302049, 1.2231828016896735, 0.6355205993310008, 1.1083550683482462, 1.3421086371559758, 0.7291648286887058, 1.2494552582851033, 0.5358424143424256, 0.39232004386853714, 0.7518072928990364]
[6.135732642575723, 7.89797791415394, 7.076044518287491, 1.3850784497201705, 2.5555103896689855, 1.1337548527657435, 2.3188036831456547, 1.0483757045195528, 2.693015091467295, 0.8935098082901933, 1.260217356323912, 1.627704733383678, 1.285336386285156, 1.039643973847359, 0.7435358822350684, 1.6463277257650408, 1.496176166349399, 0.4038864321740344, 1.2284509575332452, 1.801255593928671, 0.5316813473966296, 1.5611384424562944, 0.28712709300831973, 0.15391501682101089, 0.5652142056561775]


In [27]:
print(r2)

[0.4968306011011364, 0.3739752955805332, 0.4464963139576389, 0.8873558117386877, 0.7928509823664744, 0.8960291435148183, 0.8289705732690047, 0.912389730593018, 0.7691480257753153, 0.9237095093957393, 0.88828295101601, 0.865195857388076, 0.8682060047060998, 0.9187412052734178, 0.9293670944381128, 0.8875704494515921, 0.8739836225946033, 0.964284474461325, 0.9052659774535882, 0.8619839011324354, 0.9439889966992521, 0.8926668496153778, 0.9651834202426526, 0.9887305721593646, 0.9627474808692761]
